In [1]:
import pandas as pd
import numpy as np

import re

In [2]:
data_movie_details=pd.read_json('../Data/IMDB_movie_details.json',lines=True)
data_movie_details.head(15)

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...
2,tt0243655,"The setting is Camp Firewood, the year 1981. I...",1h 37min,"[Comedy, Romance]",6.7,2002-04-11,
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...
5,tt0286716,"Bruce Banner, a brilliant scientist with a clo...",2h 18min,"[Action, Sci-Fi]",5.7,2003-06-20,Bruce Banner (Eric Bana) is a research scienti...
6,tt0090605,57 years after Ellen Ripley had a close encoun...,2h 17min,"[Action, Adventure, Sci-Fi]",8.4,1986-07-18,"After the opening credits, we see a spacecraft..."
7,tt0243155,Bridget Jones is an average woman struggling a...,1h 37min,"[Comedy, Drama, Romance]",6.7,2001-04-13,Bridget Jones (adorably played by Renee Zellwe...
8,tt0121765,"Ten years after the invasion of Naboo, the Gal...",2h 22min,"[Action, Adventure, Fantasy]",6.6,2002-05-16,The opening crawl reveals that the Galactic Re...
9,tt0443453,Borat Sagdiyev is a TV reporter of a popular s...,1h 24min,[Comedy],7.3,2006-11-03,Borat Sagdiyev is a TV reporter of a popular s...


In [3]:
data_movie_reviews=pd.read_json('../Data/IMDB_reviews.json',lines=True)
data_movie_reviews

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"
...,...,...,...,...,...,...,...
573908,8 August 1999,tt0139239,ur0100166,False,"Go is wise, fast and pure entertainment. Assem...",10,The best teen movie of the nineties
573909,31 July 1999,tt0139239,ur0021767,False,"Well, what shall I say. this one´s fun at any ...",9,Go - see the movie
573910,20 July 1999,tt0139239,ur0392750,False,"Go is the best movie I have ever seen, and I'v...",10,It's the best movie I've ever seen
573911,11 June 1999,tt0139239,ur0349105,False,Call this 1999 teenage version of Pulp Fiction...,3,Haven't we seen this before?


In [4]:
data_movie_reviews.iloc[0][4]

'In its Oscar year, Shawshank Redemption (written and directed by Frank Darabont, after the novella Rita Hayworth and the Shawshank Redemption, by Stephen King) was nominated for seven Academy Awards, and walked away with zero. Best Picture went to Forrest Gump, while Shawshank and Pulp Fiction were "just happy to be nominated." Of course hindsight is 20/20, but while history looks back on Gump as a good film, Pulp and Redemption are remembered as some of the all-time best. Pulp, however, was a success from the word "go," making a huge splash at Cannes and making its writer-director an American master after only two films. For Andy Dufresne and Co., success didn\'t come easy. Fortunately, failure wasn\'t a life sentence.After opening on 33 screens with take of $727,327, the $25M film fell fast from theatres and finished with a mere $28.3M. The reasons for failure are many. Firstly, the title is a clunker. While iconic to fans today, in 1994, people knew not and cared not what a \'Shaws

In [5]:
df_reviews = data_movie_reviews.merge(data_movie_details, on="movie_id", how="left", suffixes=('_review','_movie'))

In [6]:
summaries = pd.Series([re.split('\s*Written by\s*\n', str(plot))[0] for plot in df_reviews['plot_summary']])
synopses = pd.Series([str(synopsis) for synopsis in df_reviews['plot_synopsis']])

In [7]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573913 entries, 0 to 573912
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   review_date     573913 non-null  object 
 1   movie_id        573913 non-null  object 
 2   user_id         573913 non-null  object 
 3   is_spoiler      573913 non-null  bool   
 4   review_text     573913 non-null  object 
 5   rating_review   573913 non-null  int64  
 6   review_summary  573913 non-null  object 
 7   plot_summary    573906 non-null  object 
 8   duration        573906 non-null  object 
 9   genre           573906 non-null  object 
 10  rating_movie    573906 non-null  float64
 11  release_date    573906 non-null  object 
 12  plot_synopsis   573906 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(10)
memory usage: 53.1+ MB


In [8]:
df_train=pd.DataFrame(columns=['movie_id','sentence_1','sentence_2','label'])

In [9]:
df_train['movie_id'] = df_reviews['movie_id']

In [10]:
df_train['sentence_1'] = pd.Series([synopses[i] if len(synopses[i])>len(summaries[i]) else summaries[i] \
                                  for i in range(len(synopses))])
df_train['sentence_2'] = df_reviews['review_summary'] + ' ' + df_reviews['review_text']
df_train['label'] = df_reviews['is_spoiler']

In [11]:
df_train

,movie_id,sentence_1,sentence_2,label
0,tt0111161,"In 1947, Andy Dufresne (Tim Robbins), a banker...",A classic piece of unforgettable film-making. ...,True
1,tt0111161,"In 1947, Andy Dufresne (Tim Robbins), a banker...",Simply amazing. The best film of the 90's. The...,True
2,tt0111161,"In 1947, Andy Dufresne (Tim Robbins), a banker...",The best story ever told on film I believe tha...,True
3,tt0111161,"In 1947, Andy Dufresne (Tim Robbins), a banker...","Busy dying or busy living? **Yes, there are SP...",True
4,tt0111161,"In 1947, Andy Dufresne (Tim Robbins), a banker...","Great story, wondrously told and acted At the ...",True
...,...,...,...,...
573908,tt0139239,"The film is told out of chronological format, ...",The best teen movie of the nineties Go is wise...,False
573909,tt0139239,"The film is told out of chronological format, ...","Go - see the movie Well, what shall I say. thi...",False
573910,tt0139239,"The film is told out of chronological format, ...",It's the best movie I've ever seen Go is the b...,False
573911,tt0139239,"The film is told out of chronological format, ...",Haven't we seen this before? Call this 1999 te...,False
